In [68]:
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

data = pd.read_csv("./data/ai_research_papers.csv")
dataset = []
for rows in data.iterrows() :
    #affiliation_country
    #affilname
    af = rows[1]['affiliation_country']
    af = set(af.split(';'))
    dataset.append(list(af))
len(dataset)

# 데이터를 원-핫 인코딩 형태로 변환
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df = df.iloc[:,1:]
frequent_itemsets = apriori(df, min_support=0.0001, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.01)


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  common_type = np.find_common_type(
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/io/parsers/c_parser_wrapper.py:365: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more info

In [69]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
nodes = frequent_itemsets[ (frequent_itemsets['length'] == 1) ].sort_values('support', ascending = False)
nodes['papers'] = len(data) * nodes['support']
nodes['itemsets'] = nodes['itemsets'].apply(lambda x: ', '.join(x))
nodes.head()

,support,itemsets,length,papers
77,0.457650,United States,1,123961.0
10,0.176952,China,1,47930.0
76,0.075647,United Kingdom,1,20490.0
21,0.074628,Germany,1,20214.0
8,0.054577,Canada,1,14783.0


In [70]:
len(nodes)

81

In [71]:
rules['anc_length'] = rules['antecedents'].apply(lambda x: len(x))
rules['cons_length'] = rules['consequents'].apply(lambda x: len(x))
edge = rules[['antecedents', 'consequents', 'antecedent support',	'consequent support','support', 'confidence', 'lift']].loc[(rules['anc_length'] == 1) & (rules['cons_length'] == 1) ].sort_values('confidence', ascending=False)
edge['antecedents'] = edge['antecedents'].apply(lambda x: ', '.join(x))
edge['consequents'] = edge['consequents'].apply(lambda x: ', '.join(x))
edge['papers'] = edge['support'] * len(data)

edge.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,papers
183,Macao,China,0.001178,0.176952,0.000713,0.605016,3.419090,193.0
176,Hong Kong,China,0.030495,0.176952,0.015491,0.507990,2.870776,4196.0
488,Qatar,United States,0.001850,0.457650,0.000853,0.461078,1.007489,231.0
414,Israel,United States,0.027058,0.457650,0.010610,0.392141,0.856857,2874.0
190,Singapore,China,0.024581,0.176952,0.008731,0.355212,2.007387,2365.0


In [72]:
len(edge)

543

In [73]:
edge['grouped_consequents'] = edge['consequents'].apply(lambda x: 'EU' if x in ['France', 'Germany', 'United Kingdom'] else x)
grouped_confidence = edge.groupby(['antecedents', 'grouped_consequents'])['confidence'].sum().reset_index(name='grouped_confidence')
grouped_confidence = grouped_confidence.loc[grouped_confidence.groupby(['antecedents'])['grouped_confidence'].idxmax()]

In [84]:
from pyvis.network import Network
import networkx as nx

G = nx.Graph()
net = Network(notebook=True)

#####전체 네트워크에서의 centrality 계산


added_nodes = set()

for idx, row in nodes.iterrows():
    color_map = {
        "United States": "#dd4b39",
        "China": "#FFFF00",
        "France": "#20B2AA",
        "United Kingdom": "#20B2AA",
        "Germany": "#20B2AA"
    }
    # 기본 색상 설정
    node_color = "#FFFFFF"  # default white if country not matched
    
    # 특정 국가에 따른 색상 설정
    if row["itemsets"] in color_map :
        node_color = color_map[row["itemsets"]]
    elif row["papers"] < 100 :
        node_color = "#FFFFFF"
    else:
        # grouped_consequents 확인하여 색상 설정
        grouped_row = grouped_confidence[grouped_confidence['antecedents'] == row["itemsets"]]
        if not grouped_row.empty:
            if grouped_row.iloc[0]["grouped_consequents"] == "United States":
                node_color = "#dd4b39"
            elif grouped_row.iloc[0]["grouped_consequents"] == "China":
                node_color = "#FFFF00"
            elif grouped_row.iloc[0]["grouped_consequents"] == "EU":
                node_color = "#20B2AA"

    # 노드 추가
    net.add_node(row["itemsets"], label=row["itemsets"], color=node_color, title="# of papers: " + str(round(row["papers"])), size=max(row["support"]*200, 10))

for idx, row in edge.iterrows():
    if row["papers"] < 0.03 :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["confidence"], width=row["confidence"], color="#DCDCDC", title = row["antecedents"] + "->" + row["consequents"] + " : "+ str(row["confidence"]))
    elif row["consequents"] == "United States" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["confidence"], width=row["confidence"]*10, arrows='to', color="#F3B0C3", title = row["antecedents"] + "->" + row["consequents"] + " : "+ str(row["confidence"]))
    elif row["consequents"] == "China" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["confidence"], width=row["confidence"]*10, arrows='to', color="#FFD700", title = row["antecedents"] + "->" + row["consequents"] + " : "+ str(row["confidence"]))
    elif row["consequents"] == "France" or row["consequents"] == "United Kingdom" or row["consequents"] == "Germany" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["confidence"], width=row["confidence"]*10, arrows='to', color="#55CBCD", title = row["antecedents"] + "->" + row["consequents"] + " : "+ str(row["confidence"]))
    else :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["confidence"], width=row["confidence"]*10, arrows='to', color="#DCDCDC", title = row["antecedents"] + "->" + row["consequents"] + " : "+ str(row["confidence"]))


net.show_buttons(filter_=['physics'])
net.show("node_paper_edge_confidence.html")


node_paper_edge_confidence.html


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.htm

In [75]:
edge['grouped_consequents'] = edge['consequents'].apply(lambda x: 'EU' if x in ['France', 'Germany', 'United Kingdom'] else x)
grouped_support = edge.groupby(['antecedents', 'grouped_consequents'])['support'].sum().reset_index(name='grouped_support')
grouped_support = grouped_support.loc[grouped_support.groupby(['antecedents'])['grouped_support'].idxmax()]

In [76]:
from pyvis.network import Network
import networkx as nx

G = nx.Graph()
net = Network(notebook=True)

#####전체 네트워크에서의 centrality 계산


added_nodes = set()

for idx, row in nodes.iterrows():
    color_map = {
        "United States": "#dd4b39",
        "China": "#FFFF00",
        "France": "#20B2AA",
        "United Kingdom": "#20B2AA",
        "Germany": "#20B2AA"
    }
    # 기본 색상 설정
    node_color = "#FFFFFF"  # default white if country not matched
    
    # 특정 국가에 따른 색상 설정
    if row["itemsets"] in color_map :
        node_color = color_map[row["itemsets"]]
    elif row["papers"] < 100 :
        node_color = "#FFFFFF"
    else:
        # grouped_consequents 확인하여 색상 설정
        grouped_row = grouped_support[grouped_support['antecedents'] == row["itemsets"]]
        if not grouped_row.empty:
            if grouped_row.iloc[0]["grouped_consequents"] == "United States":
                node_color = "#dd4b39"
            elif grouped_row.iloc[0]["grouped_consequents"] == "China":
                node_color = "#FFFF00"
            elif grouped_row.iloc[0]["grouped_consequents"] == "EU":
                node_color = "#20B2AA"

    # 노드 추가
    net.add_node(row["itemsets"], label=row["itemsets"], color=node_color, title="# of papers: " + str(row["papers"]), size=max(row["support"]*200, 10))

for idx, row in edge.iterrows():
    if row["papers"] < 10 :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, color="#DCDCDC", title = row["papers"])
    elif row["consequents"] == "United States" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, color="#F3B0C3", title = row["papers"])
    elif row["consequents"] == "China" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, color="#FFD700", title = row["papers"])
    elif row["consequents"] == "France" or row["consequents"] == "United Kingdom" or row["consequents"] == "Germany" :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, color="#55CBCD", title = row["papers"])
    else :
        net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, color="#DCDCDC", title = row["papers"])


net.show_buttons(filter_=['physics'])
net.show("node_paper_edge_paper.html")


node_paper_edge_paper.html


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.htm

In [83]:
from pyvis.network import Network
import networkx as nx

G = nx.Graph()
net = Network(notebook=True)

#####전체 네트워크에서의 centrality 계산


added_nodes = set()

for idx, row in nodes.iterrows():
    net.add_node(row["itemsets"], label=row["itemsets"], title="# of papers: " + str(row["papers"]), size=max(row["support"]*200, 10))

for idx, row in edge.iterrows():
    net.add_edge(row["antecedents"], row["consequents"], weight = row["papers"], width=row["papers"]/1000, title = row["antecedents"] + "<->" + row["consequents"] + " : "+ str(row["papers"]))


net.show_buttons(filter_=['physics'])
net.show("node_paper_edge_paper_origin.html")


node_paper_edge_paper_origin.html


/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
/home/pkmon1d/miniconda3/lib/python3.11/site-packages/pandas/core/dtypes/cast.py:1846: DeprecationWarning: np.find_common_type is deprecated.  Please use `np.result_type` or `np.promote_types`.
See https://numpy.org/devdocs/release/1.25.0-notes.html and the docs for more information.  (Deprecated NumPy 1.25)
  return np.find_common_type(types, [])  # type: ignore[arg-type]
